In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report

In [5]:
# 1. 데이터 불러오기
df = pd.read_csv("seeds_dataset.txt", delim_whitespace=True, header=None)
df.columns = ["Area", "Perimeter", "Compactness", "KernelLength", "KernelWidth",
              "AsymmetryCoeff", "KernelGrooveLength", "Target"]
df = df.dropna()

df

,Area,Perimeter,Compactness,KernelLength,KernelWidth,AsymmetryCoeff,KernelGrooveLength,Target
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1
...,...,...,...,...,...,...,...,...
205,12.19,13.20,0.8783,5.137,2.981,3.631,4.870,3
206,11.23,12.88,0.8511,5.140,2.795,4.325,5.003,3
207,13.20,13.66,0.8883,5.236,3.232,8.315,5.056,3
208,11.84,13.21,0.8521,5.175,2.836,3.598,5.044,3


In [7]:
# 2. X, y 분리 및 전처리
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].astype(int).values - 1  # 클래스 번호를 0부터 시작하도록 조정
y = to_categorical(y)  # 원-핫 인코딩

In [9]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 3. 학습/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# 4. Sequential 모델 정의
model = Sequential([
    Dense(64, activation='relu', input_shape=(7,)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
# 5. 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 6. 학습
model.fit(X_train, y_train, epochs=50, batch_size=8, validation_split=0.2, verbose=1)

Epoch 1/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5057 - loss: 0.9984 - val_accuracy: 0.9118 - val_loss: 0.6965
Epoch 2/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9023 - loss: 0.6872 - val_accuracy: 0.9412 - val_loss: 0.4769
Epoch 3/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9009 - loss: 0.5171 - val_accuracy: 0.9118 - val_loss: 0.3314
Epoch 4/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9408 - loss: 0.3720 - val_accuracy: 0.9118 - val_loss: 0.2489
Epoch 5/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9074 - loss: 0.2987 - val_accuracy: 0.9118 - val_loss: 0.2100
Epoch 6/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9388 - loss: 0.2731 - val_accuracy: 0.9118 - val_loss: 0.1795
Epoch 7/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9379 - loss: 0.2150 - val_accuracy: 0.9118 - val_loss: 0.1678
Epoch 8/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9095 - loss: 0.2526 - val_accuracy: 0.9118 - val_loss:

In [15]:
# 7. 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f"\n✅ Test Accuracy: {accuracy:.4f}")

# 8. 결과 출력
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)
print("\n📊 Classification Report:\n")
print(classification_report(y_true_classes, y_pred_classes))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9211 - loss: 0.1357

✅ Test Accuracy: 0.9286
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step

📊 Classification Report:

              precision    recall  f1-score   support

           0       0.83      0.91      0.87        11
           1       1.00      1.00      1.00        14
           2       0.94      0.88      0.91        17

    accuracy                           0.93        42
   macro avg       0.92      0.93      0.93        42
weighted avg       0.93      0.93      0.93        42

